## Day 9: Stream Processing

http://adventofcode.com/2017/day/9

### Part 1

First get rid of all the irrelevant garbage. The exclamation marked garbage needs to be removed first so garbage or group tags aren't closed prematurely, then the rest can be dealt with. If the input is correctly formed the result is a string of comma separated groups or empty strings.

In [1]:
import parsy

def not_char(c):
    return parsy.regex(r'[^' + c + r']')
    
def remove_garbage(s):
    # Remove garbage cancelled by "!"
    s = ''.join(not_char('!').many().concat().sep_by(parsy.string('!') + parsy.any_char).parse(s))
    
    # Remove the rest of the garbage
    garbage = parsy.string('<') + not_char('>').many().concat() + parsy.string('>')
    s = ''.join(not_char('<').many().concat().sep_by(garbage).parse(s))
    
    return s

In [2]:
remove_garbage('{{<ab>},{<ab>},{<ab>},{<ab>}}')

'{{},{},{},{}}'

In [3]:
remove_garbage('{{<a!>},{<a!>},{<a!>},{<ab>}}')

'{{}}'

The string should now be a tree of curly brackets, so keep note of which level we're on and add it to the final score. (The commas in the string can be ignored.)

In [4]:
def score(s):
    level = 0
    final_score = 0
    
    for c in s:
        if c == '{':
            level += 1
        elif c == '}':
            final_score += level
            level -= 1
            
    return final_score

In [5]:
def result(s):
    return score(remove_garbage(s))

In [6]:
assert result('{}') == 1
assert result('{{{}}}') == 6
assert result('{{},{}}') == 5
assert result('{{{},{},{{}}}}') == 16
assert result('{<a>,<a>,<a>,<a>}') == 1
assert result('{{<ab>},{<ab>},{<ab>},{<ab>}}') == 9
assert result('{{<!!>},{<!!>},{<!!>},{<!!>}}') == 9
assert result('{{<a!>},{<a!>},{<a!>},{<ab>}}') == 3

In [7]:
with open('input', 'r') as f:
    stream = f.read().strip()
    
result(stream)

14190

### Part 2

Modify the `remove_garbage` function to total the number of uncancelled garbage characters.

(The `>>` operator is "parse the left side but don't include it in the result" and `<<` "parse the right side but don't include it in the result".)

In [8]:
def count_garbage(s):
    # Remove garbage cancelled by "!"
    s = ''.join(not_char('!').many().concat().sep_by(parsy.string('!') + parsy.any_char).parse(s))
    
    # Count the rest of the garbage
    non_garbage = not_char('<').many()
    garbage = parsy.string('<') >> not_char('>').many().map(len) << parsy.string('>')
    return sum((non_garbage >> garbage.sep_by(non_garbage) << non_garbage).parse(s))

In [9]:
count_garbage('<{o"i!a,<{i<a>')

10

In [10]:
count_garbage(stream)

7053